In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd  drive/My\ Drive/CPD/MovieLens

/content/drive/My Drive/CPD/MovieLens


In [1]:
#### The dataset was obtained after the same preprocessing of the original MovieLens 100k dataset performed in Exponential Machines (Novikov et al. 2016)
#### to allow for direct comparison.
#### Code adapted from Exponential Machines (2016) Novikov et al.
#### url:https://github.com/Bihaqo/exp-machines

import numpy as np
import matplotlib.pyplot as plt 
from numpy import load
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from CP_Machine import CP_Machine,simple_batcher
import time
##### Load Preprocessed Data #####
x_train=load('x_train.npy')
x_test=load('x_test.npy')
y_train=load('y_train.npy')
y_test=load('y_test.npy')
num_features=26
s_features=[7,2,21,19,943,1682,10,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
##### Logistic Regression #####
oh = OneHotEncoder()
oh.fit(np.vstack((x_train, x_test))-1)
X_tr_sp = oh.transform(x_train-1)
X_te_sp = oh.transform(x_test-1)
logreg = LogisticRegression()
logreg.fit(X_tr_sp, y_train)
y_pred = logreg.predict_proba(X_te_sp)[:, 1]
print(roc_auc_score(y_test, y_pred))

##### CP_Machine #####


cp_rank = 30
num_features = len(s_features)
w_cores = [None] * num_features
begin_feature = [0] + list(np.cumsum(s_features)) #### where each feature begins in the big d vector

coef = logreg.coef_[0]
intercept = logreg.intercept_[0]

for i in range(num_features):
    local_dim = s_features[i]
 
    tmp = np.zeros((local_dim+1,cp_rank))
    tmp[0,:num_features]=1
    tmp[0,i]=intercept/num_features   
    tmp[1:s_features[i]+1,i]= coef[begin_feature[i]:begin_feature[i]+s_features[i]]

    w_cores[i] = tmp.astype(np.float32)
#

epoch_hist=[]

#### init_std=0.001, rank=30, reg=0.00005, expreg=3.6,  auc=0.7863

model = CP_Machine(rank=cp_rank, s_features=s_features, init_std=0.001, reg=0.00005, exp_reg=3.6)
model.init_from_cores(w_cores)
model.build_graph()
model.initialize_session()

start=time.time()
for epoch in range(200):
    loss_hist = []
    penalty_hist = []
    for x, y in simple_batcher(x_train, y_train, 256):
        fd = {model.X: x, model.Y: 2*y-1}
        run_ops = [model.trainer, model.outputs, model.loss, model.penalty, model.G]

        _, outs, batch_loss, penalty,weights = model.session.run(run_ops, fd)

        loss_hist.append(batch_loss)
        penalty_hist.append(penalty)
    epoch_train_loss = np.mean(loss_hist)
    epoch_train_pen = np.mean(penalty_hist)
    
#    epoch_stats = {'epoch': epoch,'train_logloss': float(epoch_train_loss)}
    epoch_stats = {'train_MSE': float(epoch_train_loss)}

     # test phase
#    if epoch%1==0 and epoch>0:
    fd = {model.X: x_test, model.Y: 2*y_test-1}
    run_ops = [model.outputs, model.loss, model.penalty, model.penalized_loss]

    outs, raw_loss, raw_penalty, loss = model.session.run(run_ops, fd)
    epoch_test_loss = roc_auc_score(y_test, outs)
    epoch_stats['test_auc'] = float(epoch_test_loss),
    #epoch_stats['penalty'] = float(raw_penalty)
    print('{}: te_auc: {:.4f}'.format(epoch, epoch_test_loss))

    epoch_hist.append(epoch_stats)
end=time.time()
pd.DataFrame(epoch_hist).plot(figsize=(8, 5))
plt.grid(True)
plt.show()
print('Training Time:{}'.format(end-start))

Instructions for updating:
non-resource variables are not supported in the long term


/opt/anaconda3/envs/Datathon2019/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/anaconda3/envs/Datathon2019/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7821230253636114
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0: te_auc: 0.7811
1: te_auc: 0.7821
2: te_auc: 0.7825
3: te_auc: 0.7827
4: te_auc: 0.7829
5: te_auc: 0.7831
6: te_auc: 0.7832
7: te_auc: 0.7833
8: te_auc: 0.7834
9: te_auc: 0.7835
10: te_auc: 0.7836
11: te_auc: 0.7837
12: te_auc: 0.7838
13: te_auc: 0.7839
14: te_auc: 0.7840
15: te_auc: 0.7841
16: te_auc: 0.7842
17: te_auc: 0.7843
18: te_auc: 0.7844
19: te_auc: 0.7845
20: te_auc: 0.7846
21: te_auc: 0.7847
22: te_auc: 0.7848
23: te_auc: 0.7849
24: te_auc: 0.7849
25: te_auc: 0.7850
26: te_auc: 0.7851
27: te_auc: 0.7851
28: te_auc: 0.7852
29: te_auc: 0.7852
30: te_auc: 0.7853
31: te_auc: 0.7853
32: te_auc: 0.7853
33: te_auc: 0.7853
34: te_auc: 0.7854
35: te_auc: 0.7854
36: te_auc: 0.7854
37: te_auc: 0.7854
38: te_auc: 0.7854
39: te_auc: 0.7854
40: te_auc: 0.7854
41: te_auc: 0.7854
42: te_auc: 0.7854
43: te_auc: 0.7854
44: te_auc: 0.7855
45: te_auc: 0.7855
46: te_auc: 0.7855
47: te_auc: 0.7855
48: t

<Figure size 800x500 with 1 Axes>

Training Time:475.1309778690338
